# Klasifikacija medicinskih snimaka pluća sa ciljem otkrivanja pneumonije

Cilj projekta je razvoj modela dubokog učenja koji automatski prepoznaje pneumoniju 
na rendgenskim snimcima pluća koristeći:
- Konvolucione neuronske mreže (CNN)
- Transfer Learning pristup (ResNet50 i Fine-Tuning)

Projekat pokazuje proces:
1. Pripreme i augmentacije podataka  
2. Treniranja modela od nule (CNN)  
3. Primjene transfer learninga (ResNet50 i Fine-Tuning)  
4. Evaluacije i poređenja rezultata  


## 1. Priprema i augmentacija podataka

PODJELA PODATAKA NA SKUPOVE:

Podaci su podijeljeni po principu 70-15-15, što podrazumijeva 70% podataka u trening skupu, namijenjenom za učenje, zatim 15% podataka u validacionom skupu, namijenjenom za praćenje tokom učenja i 15% podataka u test skupu za konačnu provjeru performansi.

AUGMENTACIJA SLIKA:

Kako bi se povećala sposobnost generalizacije modela i spriječio overfitting, tokom treniranja je uz pomoć **ImageDataGenerator** klase biblioteke Keras odrađena augmentacija.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=25,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.1,
        zoom_range=0.2,
        brightness_range=[0.8, 1.2],
        horizontal_flip=True,
        fill_mode='nearest'
    )

- rescale = 1./255 
Svaka slika ima vrijednost piksela od 0 do 255. Ovime su se vrijednosti normalizovale u opseg [0,1], a razlog je što neuronske mreže imaju tendenciju da daju bolje rezultate kada su ulazi normalizovani.
- rotation_range = 25 
Ovime se svaka slika nasumično rotira u opsegu ±25 stepeni, sa ciljem uvođenja varijabilnosti u podatke za trening, kako bi se smanjila osjetljivost modela na orijentaciju.
- width_shift_range = 0.15 
Ovime se nasumično pomijeraju slike po širini, do 15% ukupne širine slike. Time model uči da model ne mora biti savšeno centriran (u konkretnom slučaju, pacijent nije savršeno postavljen pred rendgen aparat).
- height_shift_range = 0.15 
Ovime se nasumično pomijeraju slike po visini do 15%. Svrha je ista kao i kod prethodnog, da model ne zavisi od pozicije  pluća na slici. 
- shear_range = 0.1
Ovime se postiže blaga deformacija oblika, pa se dodatno povećava raznolikost ulaznih podataka.
- zoom_range = 0.2 
Ovime se slika umanjuje ili uvećava do 20%, sa ciljem simuliranja blizine pacijenta od rendgen aparata. 
- brightness_range = [0.8, 1.2] 
Ovime se mijenja osvjetljenje slike, čineći je nasumično svjetlijom ili tamnijom, što pomaže modelu da nauči da zanemari promjene u osvjetljenju i da se fokusira na strukturalne obrasce.
- horizontal_flip = True  
Ovime se, budući da rendgenski snimak nema horizontalnu asimetriju, postiže raznovrsnost, bez gubitka značenja.
- fill_mode = ‘nearest’ 
Kada se slike zumiraju, pomijeraju ili rotiraju, može doći do pojave ,,praznih’’ oblasti, pa se s tim u vezi koristi ova opcija, kako bi se te praznine popunile vrijednostima najbližih piksela.

Ove transformacije omogućavaju da model vidi različite varijacije istih snimaka i bolje nauči kako pneumonija može izgledati u različitim slučajevima.


## 2. Kreiranje modela od nule (CNN)
U prvom dijelu projekta razvijen je model konvolucione neuronske mreže CNN, koji za zadatak ima klasifikaciju rendgen snimaka u dvije kategorije: NORMAL i PNEUMONIA. CNN modeli su izuzetno pogodni za obradu slika jer automatski uče karakteristike iz piksela slike. 
Glavni dio u izgradnji jeste `build_cnn(image_size)`.

In [ ]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import Adam

def build_cnn(image_size):
    model = models.Sequential([
        layers.Input(shape=(*image_size, 3)),

        layers.Conv2D(32, (3,3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(64, (3,3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(128, (3,3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(256, (3,3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Flatten(),

        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),

        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.5), 

        layers.Dense(1, activation='sigmoid')
    ])

    optimizer = Adam(learning_rate=5e-5)  

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

Cilj funkcije je da se od nule izgradi kovulaciona neuronska mreža sposobna da samostalno nauči vizuelne obrasce karakteristične za zdrava i oboljela pluća. Ideja je da model postepeno ,,uči’’ od jedostavnih ka složenim karakteristikama, kroz više hijerarhijskih slojeva konvoluacije.

Logika iza strukture modela:

1.	**Ulazni sloj:** Mreža prima slike veličine 224x224x3 (RGB format, iako su rendgenske slike monohromatske, ali se radi konverzija radi kompatibilnosti sa Keras API-jem). 
2.	**Prvi konvoluacicioni sloj:**  U prvom sloju se koristi 32 filtera dimenzije 3x3. Ovaj sloj detektuje osnovne oblike i ivice u slikama, što je prva faza u ,,vizuelnom učenju’’.  BatchNormalization() tehnikom noramalizacije se postiže brže i stabilnije treniranje. MaxPooling2D((2,2)) redukuje prostorne dimenzije slike. 
3.	**Naredni slojevi:** Svaki sledeći sloj povećava broj filtera (64, 128, 256), čime mreža uči sve složenije vizuelne obrasce: konture pluća, obrasce zadebljanja, promjene teksture i kontrasta unutar tkiva. ReLU se koristi za uvođenje nelinearnosti i poboljšavanje brzine konvergencije. 
4.	**Flatten sloj:** Nakon što su sve prostorne karakteristike ,,izvučene’’, ovaj sloj pretvara podatke u jedno-dimenzionalni niz, pogodan za ulaz u Dense slojeve.
5.	**Dense slojevi:** Prvi Dense sloj ima 256 neurona i koristi L2 regulaciju da ograniči težine i na taj način spriječi overfitting.  Nakon toga slijedi BatchNormalization i Dropout(0.5) koji nasumično isključuje 50% neurona, sa ciljem poboljšavanja sposobnosti generalizacije. Sledeći Dense sloj (sa 128 neurona) dodatno prorjeđuje mrežu prije konačne odluke. 
6.	**Izlazni sloj:** Sigmod funkcija daje rezultat u opsegu [0,1], koji se tumači kao vjerovatnoća da slika prikazuje pluća pacijenta sa pneumonijom. 

Način na koji će mreža učiti:

- Optimizacija – Adam optimizer:
Za treniranje modela korišćen je Adam optimizator, jer automatski prilagođava korake učenja tokom treniranja i dobro funkcioniše i na velikim i na neuravnoteženim skupovima. U ovom slučaju korišćen je learning_rate vrijednosti 5e-5. Razlog odabira relativno male vrijednosti je eksperimentalno utvrđen, jer mali learning_rate omogućava postepenije i u ovom slučaju preciznije učenje. 
- Gubitak (loss):
Pošto se radi o binarnoj klasifikaciji (NORMAL/PNEUMONIA), korišćena je binary crossentropy funkcija gubitka. Ona mjeri koliko su predikcije modela udaljene od stvarnih oznaka. Što je loss manji – to je model bolji. 
- Metrika – Accuracy:
Kao dodatna mjera praćenja performansi koristi se tačnost (accuracy), koja pokazuje procenat ispravno klasifikovanih slika tokom treninga i validacije.


Proces treniranja modela
Nakon definisanja arhitekture konvulacione neuronske mreže, slijedi faza treniranja (učenja) modela nad pripremljenim skupom podataka. Cilj procesa jeste da model optimizuje svoje parametre tako da minimizuje grešku između stvarnih oznaka (NORMAL/PNEUMONIA) i svojih predviđanja. 
U okviru koda, treniranje se odvija u funkciji `train_model(model, train_gen, val_gen, epochs)` a ključni su balansiranje klasa, definisanje callback mehanizma i sprovođenje samog učenja putem model.fit(). 

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np

def train_model(model, train_gen, val_gen, epochs):
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(train_gen.classes),
        y=train_gen.classes
    )
    class_weights = dict(enumerate(class_weights))
    print("Class Weights:", class_weights)

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
        ModelCheckpoint('best_pneumonia_model.keras', save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-6)
    ]

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=callbacks,
        class_weight=class_weights
    )
    return history

- **Balansiranje klasa – Class Weights:**
Da bi se ublažila pristrasnost uzrokovana neuravnoteženim klasama, u toku treniranja modela primjenjen je pristup class_weight. Funkcija compute_class_weight iz sklearn.utils izračunava težinu za svaku klasu tako da rijeđa klasa dobija veću težinu pri ažuriranju grešalka u toku treninga, kako bi svaka greška doprinjela gubitku proporcionalno zastupljenosti svoje klase. 

- **Callback:**
Callback funkcije prate proces treniranja i automatski reaguju kada se ispune određeni uslovi. 
1. EarlyStopping zaustavlja treniranje ako se validacioni gubitak ne poboljša tokom 8 uzastopnih epoha. Time se sprječava overfitting. Parametar restore_best_weights = True vraća mrežu na stanje u kojem je imala najbolje rezultate na validacionom skupu. 
2. ModelCheckpoint čuva model svaki put kada dostigne najbolju validacionu tačnost. Time se obezbjeđuje da čak i ako kasnije epohe pogoršaju rezultate, najbolja verzija modela ostaje sačuvana kao best_pneumonia_model.keras. 
3. ReduceLROnPlateau automatski smanjuje learning rate za 30% ako validacioni gubitak stagnira tokom dvije epohe. To omogućava finije prilagođavanje težina u kasnijim fazama treniranja, kada se model već približio optimumu. 
- **Proces treniranja:**
Na kraju, treniranje se sprovodi pozivom model.fit. Ova funkcija prolazi kroz sve slike u trening skupu u više epoha (do 30 puta), dok u svakoj epohi: koristi train_gen za učenje na augmentovanim podacima, provjerava performanse na val_gen skupu i automatski primjenjuje callback mehanizme.


## 3. Primjene transfer learninga (ResNet50 i Fine-Tuning)  
Naredni korak, u cilju poboljšanja rezultata pri dijagnostifikovanja pneumonije, jeste primjena Transfer Learning – tehnike učenja pri kojoj se koristi unaprijed istreniran model na velikom skupu podataka, i taj model se prilagođava za konkretni zadatak. 
Treniranje dubokih neuronskih mreža od nule zahtijeva ogroman broj podataka i računarske resurse. Transfer learning omogućava iskorišćavanje prethodno naučenih vizuelnih karakteristika (ivica, oblika, tekstura), brže konvergiranje modela i bolju generalizaciju. 

Za osnovu modela korišćena je arhitektura ResNet50. 

ResNet50 arhitektura poznata je po korišćenju ,,residual connections” sa ciljem rješavanja problema degradacije u dubokim neuronskim mrežama, na način da omogućavaju direktan protok informacija kroz tzv. ,,skip’’ konekcije.


**Prva faza:**

U prvoj fazi transfer learning pristupa model ResNet50 se koristi kao feature extraction, što znači da se svi njegovi slojevi ,,zamrzavaju’’ i koristi se samo njegova sposobnost da prepozna opšte vizuelne obrasce: base_model.trainable = False. Dakle, osnovna mreža sada sadrži karakteristike koje su generalno korisne za različite specifične zadatke. 

In [ ]:
from tensorflow.keras.applications import ResNet50

def build_resnet_transfer(image_size):
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(*image_size, 3)
    )
    base_model.trainable = False  

    inputs = layers.Input(shape=(*image_size, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=inputs, outputs=outputs)

    optimizer = Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


Ovi slojevi predstavljaju novu glavu mreže, koja ima zadatak da koristi već izdvojene karakteristike i donese konačnu odluku – PNEUMONIA ili NORMAL. Tokom treniranja se takođe koriste EarlyStopping, ModelCheckpoint, ReduceLROnPlateau. 
Cilj ove faze jeste isključivo prilagođavanje novih slojeva na vrhu mreže – dok se osnovni slojevi ResNet-a ne mijenjaju. Model uči kako da koristi već izdvojene opšte karakteristike slike za prepoznavanje konkretnih obrazaca na rendgenskim snimcima pluća. 


**Druga faza:**

Druga faza jeste fine-tuning ResNet50 modela, sa ciljem poboljšanja rezultata dijagnostike. Učitava se prethodno sačuvan model best_pneumonia_resnet.keras i otključava posljednjih 30 slojeva u ResNet50 mreži. Cilj je da se najdublji slojevi dodatno prilagode medicinskim karakteristikama snimaka, jer oni uče najapstraktnije obrasce. 
Koristi se mnogo manji learning rate (5e-6) jer model već ima stabilne težine i sada su potrebne suptilne korekcije. Ponovo se primjenjuju callback funkcije, sada sa manjim patience parametrima. 
Cilj jeste da se poboljša preciznost i da sada model ,,nauči” da razlikuje fine nijanse u teksturi i kontrastu plućnog tkiva, koje su ključne za prepoznavanje pneumonije. 


In [ ]:
from tensorflow.keras.models import load_model

def fine_tune_resnet(model_path):
    model = load_model(model_path)

    base_model = None
    for layer in model.layers:
        if isinstance(layer, models.Model):
            base_model = layer
            break

    if base_model is None:
        raise ValueError("Base model (ResNet) not found!")

    for layer in base_model.layers[:-30]:
        layer.trainable = False
    for layer in base_model.layers[-30:]:
        layer.trainable = True

    model.compile(optimizer=Adam(learning_rate=5e-6),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    return model

Dvostepeni pristup (feature ectraction + fine-tuning) pokazao se efikasnim jer kombinuje brzu konvergenciju i specifično prilagođavanje.

## 4. Evaluacija i poređenje rezultata

Nakon treninga sva tri modela (CNN od nule, ResNet50 i fine-tuned ResNet50), sprovedena je evaluacija na nezavisnom test skupu pomoću metrika test accuracy, test loss, AUC score, F1-score.

Za evaluaciju modela koristila se `evaluate_model(model, test_gen)` funkcija. Ova funkcija služi za **procjenu performansi modela na test skupu** i prikaz glavnih metrika i grafika.  

- Evaluacija modela pomoću `model.evaluate(test_gen)` izračunava ukupni *test loss* i *test accuracy*
- Predikcija klasa pomoću `model.predict()` dobija se vjerovatnoća da slika prikazuje pneumoniju.
- Izvještaj o klasifikaciji - ispisuje se `classification_report`
- Confusion Matriks prikazuje broj ispravnih i pogrešnih predikcija po klasama.  
- ROC kriva i AUC vrijednost - ROC (Receiver Operating Characteristic) kriva prikazuje kompromis između *true positive rate* i *false positive rate*, što je kriva bliža gornjem lijevom uglu, model je bolji. AUC (Area Under Curve) pokazuje ukupnu sposobnost modela da razlikuje dvije klase (bliže 1 → bolji model).  


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate_model(model, test_gen):
    test_loss, test_acc = model.evaluate(test_gen)
    print(f"\n Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

    y_pred = model.predict(test_gen)
    y_pred_classes = (y_pred > 0.5).astype("int32").flatten()
    y_true = test_gen.classes

    print("\n Classification Report:")
    print(classification_report(y_true, y_pred_classes, target_names=list(test_gen.class_indices.keys())))

    cm = confusion_matrix(y_true, y_pred_classes)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=test_gen.class_indices.keys(),
                yticklabels=test_gen.class_indices.keys())
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()

    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(6,5))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f"ROC Curve (AUC = {roc_auc:.3f})")
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC Curve)')
    plt.legend(loc="lower right")
    plt.show()

    print(f" AUC Score: {roc_auc:.3f}")